In [ ]:
#IMPORTS OF PACKAGES
import mne
from os.path import join
import matplotlib.pyplot as plt
import pathlib

plt.ion() #toggle interactive plotting
# import numpy as np

In [ ]:
# PATHS
path = pathlib.Path.cwd()
meg_path = path.parents[2] / "834761" / "0114" / "20230927_000000" / "MEG" / "001.self_block1" / "files"

bem_path = path.parents[2] / "835482" / "0114" / "bem"

subjects_dir = path.parents[2] / "835482" 

raw_name = 'self_block1.fif'
fwd_name = 'self_block1-oct-6-src-5120-fwd.fif'

In [ ]:
raw = mne.io.read_raw_fif(join(meg_path, raw_name), preload=True)
raw.plot() ## what happens after 10 seconds?
raw.compute_psd(n_jobs=-1).plot()
raw.compute_psd(n_jobs=-1, tmax=9).plot()